# Assignment Optimization Model

This notebook implements a **minimum-cost assignment problem** using
binary decision variables and a mixed-integer linear programming (MILP)
formulation.

The model is solved using **PuLP** with the open-source **CBC solver**.


## Sets

- $W$: set of workers, indexed by $w$
- $J$: set of jobs, indexed by $j$

## Parameters

- $c_{w,j}$: cost of assigning worker $w \in W$ to job $j \in J$

## Decision Variables

$$
x_{w,j} =
\begin{cases}
1 & \text{if worker } w \text{ is assigned to job } j \\
0 & \text{otherwise}
\end{cases}
\quad \forall w \in W, \; j \in J
$$


## Objective Function

Minimize total assignment cost:

$$
\min \sum_{w \in W} \sum_{j \in J} c_{w,j} \, x_{w,j}
$$

## Constraints

### Worker assignment constraints

Each worker is assigned to exactly one job:

$$
\sum_{j \in J} x_{w,j} = 1 \quad \forall w \in W
$$

### Job assignment constraints

Each job is assigned to exactly one worker:

$$
\sum_{w \in W} x_{w,j} = 1 \quad \forall j \in J
$$

### Binary constraints

$$
x_{w,j} \in \{0,1\} \quad \forall w \in W, \; j \in J
$$

## Notes

- The base model assumes a **square assignment** problem where $|W| = |J|$.
- Extensions such as forbidden assignments or penalty costs can be added
  without changing the core structure.


In [42]:
# Imports

from __future__ import annotations 
from typing import Dict, Tuple, List

from pulp import( LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD, LpStatus)

In [43]:
Cost= Dict[Tuple[str, str], float]

In [44]:
def solve_assignment(cost: Cost, forbidden: List[Tuple[str,str]] | None=None):
    
    """
        Classic square assignment problem:
            – Each worker must be assigned to exactly one job
            – Each job also assigned to exactly one worker
    """
    forbidden=forbidden or []
    
    workers= sorted({a for (a,b) in cost.keys()})
    jobs = sorted({b for (a,b) in cost.keys()})
    
    
    if len(workers) != len(jobs):
        raise ValueError(f"The square assignment optimization model requires same number of jobs and works."
                         f"Got Workers={len(workers)} jobs={len(jobs)}")
        
    model= LpProblem("Assignment", LpMinimize)
    
    X = LpVariable.dicts("X", (workers, jobs), 0, 1, LpBinary)
    
    model += (lpSum(cost[(w,j)]* X[w][j] for w in workers for j in jobs), "TotalCost")
    
    for a in workers:
        model += (lpSum(X[a][b] for b in jobs ) ==1, f"AssigEachWorker_{a}")
    
    for b in jobs:
        model += (lpSum(X[a][b] for a in workers) ==1 , f"AssignEacjJob_{b}")
        
    for (a,b) in forbidden:
        if a in workers and b in jobs:
            model += (X[a][b] ==0 , f"Forbid_{a}_{b}")
    
    
    model.solve(PULP_CBC_CMD(msg=False))
    
    status = LpStatus[model.status]
    obj = float(model.objective.value()) if model.objective is not None else float("nan")
    
    assignments = []
    
    for w in workers:
        for j in jobs:
            if X[w][j].value() ==1:
                assignments.append((w,j, cost[(w,j)]))
                
    return status, obj, assignments
       
            

#### sample data and run opt model

In [45]:
# Note: Right now I am forbidden worker C assigned to Job3

cost_matrix: Cost= {
    
    ("A", "Job1"): 9,  ("A", "Job2"): 2,  ("A", "Job3"): 7,
    ("B", "Job1"): 6,  ("B", "Job2"): 4,  ("B", "Job3"): 3,
    ("C", "Job1"): 5,  ("C", "Job2"): 8,  ("C", "Job3"): 1,
}

status, obj, assignments = solve_assignment(cost_matrix, forbidden=[("C", "Job3")])
status, obj, assignments

('Optimal', 10.0, [('A', 'Job2', 2), ('B', 'Job3', 3), ('C', 'Job1', 5)])

In [46]:
print("Optmodel Status:", status)
print("Total cost:" , obj)
print("Assignments:")
for w, j, c in assignments:
    print(f" {w} --> {j} (cost={c})")
    
    
        


Optmodel Status: Optimal
Total cost: 10.0
Assignments:
 A --> Job2 (cost=2)
 B --> Job3 (cost=3)
 C --> Job1 (cost=5)
